In [18]:
import spotipy
import pandas as pd
import numpy as np

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

cid = os.getenv("SPOTIFY_CLIENT_ID")
secret = os.getenv("SPOTIFY_CLIENT_SECRET")

from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [20]:
# Get Billie Eilish's discography

artist_uri = 'spotify:artist:6qqNVTkY8uBg9cP3Jd7DAH'
album_results = sp.artist_albums(artist_uri, album_type='album')
albums = album_results['items']
while album_results['next']:
    album_results = sp.next(album_results)
    albums.extend(album_results['items'])

In [21]:
print("Billie Eilish's discography:")
for album in albums:
    print(album['name'])

Billie Eilish's discography:
HIT ME HARD AND SOFT
Happier Than Ever
WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?


In [26]:
# Create dataframe for each song of each album

songs = []
for album in albums:
    album_uri = album['uri']
    tracks = sp.album_tracks(album_uri)['items']
    for track in tracks:
        song = {}
        # song metadata
        song['name'] = track['name']
        song['track_number'] = track['track_number']
        song['album'] = album['name']
        song['release_date'] = album['release_date']

        track_uri = track['uri']
        # song features
        features = sp.audio_features(track_uri)[0]
        song['danceability'] = features['danceability']
        song['energy'] = features['energy']
        song['key'] = 'minor' if features['mode'] == 0 else 'major'
        song['loudness'] = features['loudness']
        song['mode'] = features['mode']
        song['speechiness'] = features['speechiness']
        song['acousticness'] = features['acousticness']
        song['instrumentalness'] = features['instrumentalness']
        song['liveness'] = features['liveness']
        song['valence'] = features['valence']
        song['tempo'] = features['tempo']

        songs.append(song)

df = pd.DataFrame(songs)
df['release_date'] = pd.to_datetime(df['release_date'])
df['year'] = df['release_date'].dt.year

In [29]:
df.to_csv('billie_eilish_discography.csv', index=False)